In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Fine-tuning a model on a translation task

In [2]:
! pip  install subword-nmt
! pip install nltk
! pip install torchtext==0.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torchtext==0.11.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2)
ERROR: No matching distribution found for torchtext==0.11.0


In [3]:
import pandas as pd
import nltk

In [4]:
phoenix_dataset_dev = '/content/drive/MyDrive/Colab Notebooks/PHOENIX-2014-T.dev.corpus.csv'
phoenix_dataset_test = '/content/drive/MyDrive/Colab Notebooks/PHOENIX-2014-T.test.corpus.csv'
phoenix_dataset_train = '/content/drive/MyDrive/Colab Notebooks/PHOENIX-2014-T.train.corpus.csv'
# Load the Phoenix dataset
phoenix_dev = pd.read_csv(phoenix_dataset_dev, sep="|")
phoenix_test = pd.read_csv(phoenix_dataset_test, sep="|")
phoenix_train = pd.read_csv(phoenix_dataset_train, sep="|")


In [5]:
phoenix_all = pd.concat([phoenix_dev, phoenix_test, phoenix_train])

In [6]:
phoenix_all.shape

(8257, 7)

In [7]:
phoenix_all.head(5)

,name,video,start,end,speaker,orth,translation
0,11August_2010_Wednesday_tagesschau-2,11August_2010_Wednesday_tagesschau-2/1/*.png,-1,-1,Signer08,DRUCK TIEF KOMMEN,tiefer luftdruck bestimmt in den nächsten tage...
1,11August_2010_Wednesday_tagesschau-3,11August_2010_Wednesday_tagesschau-3/1/*.png,-1,-1,Signer08,ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWIT...,das bedeutet viele wolken und immer wieder zum...
2,11August_2010_Wednesday_tagesschau-8,11August_2010_Wednesday_tagesschau-8/1/*.png,-1,-1,Signer08,WIND MAESSIG SCHWACH REGION WENN GEWITTER WIND...,meist weht nur ein schwacher wind aus untersch...
3,25October_2010_Monday_tagesschau-22,25October_2010_Monday_tagesschau-22/1/*.png,-1,-1,Signer01,MITTWOCH REGEN KOENNEN NORDWEST WAHRSCHEINLICH...,am mittwoch hier und da nieselregen in der nor...
4,05May_2011_Thursday_tagesschau-25,05May_2011_Thursday_tagesschau-25/1/*.png,-1,-1,Signer08,JETZT WETTER WIE-AUSSEHEN MORGEN FREITAG SECHS...,und nun die wettervorhersage für morgen freita...


In [8]:
phoenix_need = phoenix_all[['translation', 'orth']]

In [9]:
phoenix_need.head(5)

,translation,orth
0,tiefer luftdruck bestimmt in den nächsten tage...,DRUCK TIEF KOMMEN
1,das bedeutet viele wolken und immer wieder zum...,ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWIT...
2,meist weht nur ein schwacher wind aus untersch...,WIND MAESSIG SCHWACH REGION WENN GEWITTER WIND...
3,am mittwoch hier und da nieselregen in der nor...,MITTWOCH REGEN KOENNEN NORDWEST WAHRSCHEINLICH...
4,und nun die wettervorhersage für morgen freita...,JETZT WETTER WIE-AUSSEHEN MORGEN FREITAG SECHS...


In [10]:
phoenix_need = phoenix_need.rename(columns={"translation": "src", "orth": "trg"})


In [11]:
!pip install datasets transformers[sentencepiece] sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [13]:
import transformers

print(transformers.__version__)

4.29.2


In [14]:
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="pytorch")

This notebook is built to run with any model checkpoint from the Model Hub as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the mariav/helsinki-opus-de-en-fine-tuned-wmt16 checkpoint.

In [15]:
model_checkpoint = "mariav/helsinki-opus-de-en-fine-tuned-wmt16"

In [16]:
!pip install rouge_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from datasets import load_metric


rouge = load_metric("rouge")
bleu = load_metric("sacrebleu")
meteor = load_metric("meteor")

<ipython-input-17-96c6296b0d6b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Prepare the dataset to the dict form. The dataset object itself is DatasetDict, which contains one key for the training, validation and test set:

In [18]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

phoenix_df = phoenix_need

train_df, test_df = train_test_split(phoenix_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

train_df = train_df.rename(columns={'translation': 'src', 'trg': 'trg'})
val_df = val_df.rename(columns={'translation': 'src', 'trg': 'trg'})
test_df = test_df.rename(columns={'translation': 'src', 'trg': 'trg'})

train_df1 = list()
train_df1.append({'translation' : train_df.to_dict('records')})
val_df1 = list()
val_df1.append({'translation' : val_df.to_dict('records')})
test_df1 = list()
test_df1.append({'translation' : test_df.to_dict('records')})

train_df2 = pd.DataFrame(train_df1)
val_df2 = pd.DataFrame(val_df1)
test_df2 = pd.DataFrame(test_df1)

train_df2 = train_df2.explode('translation')
val_df2 = val_df2.explode('translation')
test_df2 = test_df2.explode('translation')

train_df2 = train_df2.reset_index(drop=True)
val_df2 = val_df2.reset_index(drop=True)
test_df2 = test_df2.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df2)
val_dataset = Dataset.from_pandas(val_df2)
test_dataset = Dataset.from_pandas(test_df2)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

dataset_dict.save_to_disk("phoenix_dataset")


Saving the dataset (0/1 shards):   0%|          | 0/5944 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/661 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1652 [00:00<?, ? examples/s]

In [19]:
raw_datasets = dataset_dict

In [20]:
!pip install sentencepiece==0.1.96

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!pip install transformers[sentencepiece] datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Before we can feed those texts above to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will  tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.
To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [23]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


We feed our samples to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.


In [24]:
max_input_length = 90
max_target_length = 90
source_lang = "src"
target_lang = "trg"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
prefix = ""

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [25]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/5944 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/661 [00:00<?, ? examples/s]

Map:   0%|          | 0/1652 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [70]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load pre-trained model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_tf=True)

# Set encoder to eval mode
model.model.encoder.eval()


All TF 2.0 model weights were used when initializing MarianMTModel.

All the weights of MarianMTModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MarianMTModel for predictions without further training.


MarianEncoder(
  (embed_tokens): Embedding(58101, 512, padding_idx=58100)
  (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
  (layers): ModuleList(
    (0-5): 6 x MarianEncoderLayer(
      (self_attn): MarianAttention(
        (k_proj): Linear(in_features=512, out_features=512, bias=True)
        (v_proj): Linear(in_features=512, out_features=512, bias=True)
        (q_proj): Linear(in_features=512, out_features=512, bias=True)
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation_fn): SiLUActivation()
      (fc1): Linear(in_features=512, out_features=2048, bias=True)
      (fc2): Linear(in_features=2048, out_features=512, bias=True)
      (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
  )
)

In [27]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Note that we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.
To instantiate a Seq2SeqTrainer, we will need to define three more things. The most important is the Seq2SeqTrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [74]:
batch_size = 7
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=7,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    seed=7575
)

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [75]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [76]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    bleu_labels = [[label] for label in labels]

    # rougeLSum expects newline after each sentence
    rouge_preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    rouge_labels = ["\n".join(nltk.sent_tokenize(label))
                      for label in labels]

    return {
        "bleu": [preds, bleu_labels],
        "meteor": [preds, labels],
        "rouge": [rouge_preds, rouge_labels]
    }

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(predictions=decoded["rouge"][0], references=decoded["rouge"][1], use_stemmer=True)
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    bleu_result = bleu.compute(predictions=decoded["bleu"][0], references=decoded["bleu"][1])
    result["bleu"] = bleu_result["score"]

    meteor_result = meteor.compute(predictions=decoded["meteor"][0], references=decoded["meteor"][1])
    result["meteor"] = meteor_result["meteor"]

    return result


To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up here if you haven't already!) then execute the following cell and input your username and password:

In [45]:
from huggingface_hub import notebook_login

notebook_login()

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [77]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/content/helsinki-opus-de-en-fine-tuned-wmt16-finetuned-src-to-trg is already a clone of https://huggingface.co/musralina/helsinki-opus-de-en-fine-tuned-wmt16-finetuned-src-to-trg. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the train method:

In [78]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Meteor
1,1.684500,0.970114,62.267300,30.794500,59.220200,59.245700,23.287400,21.947627,0.511476
2,0.912400,0.885342,63.276600,31.380200,59.971800,59.954600,22.816900,22.134896,0.518350
3,0.756600,0.863591,63.746300,30.932300,60.366500,60.310700,23.824500,22.383399,0.521132
4,0.672300,0.869742,63.900500,31.011700,60.596900,60.568500,23.184600,22.340561,0.527967
5,0.586200,0.866391,64.733200,32.214200,61.275500,61.281600,22.800300,23.866465,0.535914
6,0.520900,0.892214,64.117600,31.290100,60.565300,60.597500,23.987900,22.854210,0.531777
7,0.480600,0.893418,64.290000,31.954200,60.994900,60.988100,23.676200,23.612529,0.536135


TrainOutput(global_step=5950, training_loss=0.7654132567333574, metrics={'train_runtime': 1088.9635, 'train_samples_per_second': 38.209, 'train_steps_per_second': 5.464, 'total_flos': 368959867453440.0, 'train_loss': 0.7654132567333574, 'epoch': 7.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [79]:
trainer.push_to_hub()

Upload file runs/May31_05-46-18_39471e0edb5a/events.out.tfevents.1685511994.39471e0edb5a.4025.8:   0%|        …

To https://huggingface.co/musralina/helsinki-opus-de-en-fine-tuned-wmt16-finetuned-src-to-trg
   0cf20de..a5db2f0  main -> main

   0cf20de..a5db2f0  main -> main



'https://huggingface.co/musralina/helsinki-opus-de-en-fine-tuned-wmt16-finetuned-src-to-trg/commit/a5db2f0c0d424c722a1b8ca730d856c88aa4d652'